# Chap 10

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pystan

In [3]:
df = pd.read_csv('./input/data-category.txt')
print(df.shape)
display(df.head())

(300, 4)


,Age,Sex,Income,Y
0,18,1,472,2
1,18,0,468,5
2,18,1,451,6
3,18,1,441,6
4,18,1,499,6


In [4]:
model_102 = pystan.StanModel('./model/model10-2.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f32c2720a3b7d0bd38d561cae1a67263 NOW.


In [11]:
X = np.concatenate([np.ones((df.shape[0], 1)), df.drop('Y', axis=1).values], axis=1)
X[:, 1] = X[:, 1] / 100.0
X[:, 3] = X[:, 3] / 1000.0
stan_data = {
    'N': df.shape[0],
    'D': df.shape[1],
    'K': df['Y'].nunique(),
    'X': X,
    'Y': df['Y'].values,
}
fit_102 = model_102.sampling(data=stan_data, seed=1234)
fit_102

Inference for Stan model: anon_model_f32c2720a3b7d0bd38d561cae1a67263.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
b_raw[0,0]  -0.13    0.03   1.07  -2.25  -0.88  -0.13   0.59   1.94   1493    1.0
b_raw[1,0]  -1.52    0.03   1.57  -4.59  -2.58  -1.51  -0.49   1.58   2089    1.0
b_raw[2,0]  -1.22  6.9e-3   0.36  -1.93  -1.47  -1.22  -0.98   -0.5   2759    1.0
b_raw[3,0]   2.82    0.04   1.59  -0.33   1.75   2.84   3.86   5.98   2054    1.0
b_raw[0,1]   0.24    0.04   1.77  -3.35  -0.97   0.23    1.5   3.55   1859    1.0
b_raw[1,1]  -4.76    0.05   2.95 -10.74  -6.71  -4.75  -2.72   0.95   4000    1.0
b_raw[2,1]  -3.35    0.01   0.89  -5.34  -3.89  -3.29  -2.71  -1.81   4000    1.0
b_raw[3,1]   2.11    0.05   2.69  -3.24   0.27   2.14   3.97   7.29   2614    1.0
b_raw[0,2]  -0.95    0.03    1.2  -3.33  -1.75  -0.94  -0.12   1.35   154

In [12]:
df = pd.read_csv('./input/data-usagitokame.txt')
print(df.shape)
df.head()

(30, 2)


,Loser,Winner
0,1,2
1,1,2
2,2,1
3,1,2
4,1,2


In [31]:
np.random.seed(123)
G = 30
mu_pf = [0, 1.5]
df_sim = pd.DataFrame({
    '0': np.random.normal(mu_pf[0], 1.0, G),
    '1': np.random.normal(mu_pf[1], 1.0, G),
})
df_sim['Loser'] = (df_sim['0'] > df_sim['1']).astype(int) + 1
df_sim['Winner'] = (df_sim['0'] <= df_sim['1']).astype(int) + 1
df_sim.head()

,0,1,Loser,Winner
0,-1.085631,1.244381,1,2
1,0.997345,-1.298589,2,1
2,0.282978,-0.271533,2,1
3,-1.506295,0.800123,1,2
4,-0.578600,2.427462,1,2


In [18]:
model_103 = pystan.StanModel('./model/model10-3.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5e301884ecf94b1b768e5b89d4674227 NOW.


In [32]:
stan_data = {
    'N': 2,
    'G': df.shape[0],
    'LW': df[['Loser', 'Winner']].values,
}
fit_103 = model_103.sampling(data=stan_data, seed=1234)
fit_103

Inference for Stan model: anon_model_5e301884ecf94b1b768e5b89d4674227.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                    mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
performance[0,0]   -0.18    0.01   0.91  -2.04  -0.77  -0.16   0.43   1.56   4000    1.0
performance[1,0]   -0.18    0.01   0.91  -1.97  -0.77  -0.15   0.43   1.56   4000    1.0
performance[2,0]    0.46    0.01   0.83  -1.14  -0.09   0.47   1.03   2.11   4000    1.0
performance[3,0]    -0.2    0.01   0.91  -2.04  -0.81  -0.19   0.42   1.48   4000    1.0
performance[4,0]   -0.19    0.01   0.92  -2.01   -0.8  -0.17   0.47   1.56   4000    1.0
performance[5,0]   -0.18    0.01   0.93   -2.1   -0.8  -0.16   0.45   1.57   4000    1.0
performance[6,0]   -0.18    0.01   0.91  -2.07  -0.78  -0.15   0.45   1.54   4000    1.0
performance[7,0]   -0.19    0.01   0.92  -2.06  -0.82  -0.16   0.45   1.49   4000    1.0
performanc

In [3]:
df = pd.read_csv('./input/data-shogi-player.txt')
print(df.shape)
df.head()

(6231, 2)


,Loser,Winner
0,22,83
1,39,27
2,47,105
3,1,7
4,45,81


In [4]:
model_104 = pystan.StanModel('./model/model10-4.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_fb2565cdad3dc7aa44fc079adb816747 NOW.


In [ ]:
stan_data = {
    'N': np.max(df.values),
    'G': df.shape[0],
    'LW': df[['Loser', 'Winner']].values,
}
fit_104 = model_104.sampling(data=stan_data, pars=['mu','s_mu','s_pf'], seed=1234)
fit_104

In [5]:
df = pd.read_csv('../chap08/input/data-salary-2.txt')
print(df.shape)
df.head()

(40, 3)


,X,Y,KID
0,7,457,1
1,10,482,1
2,16,518,1
3,25,535,1
4,5,427,1


In [6]:
model_105 = pystan.StanModel('./model/model10-5.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_acf74524cab145ec054fd26d4eb66f87 NOW.


In [7]:
stan_data = {
    'N': df.shape[0],
    'K': df['KID'].nunique(),
    'X': df['X'].values,
    'Y': df['Y'].values,
    'KID': df['KID'].values,
}
fit_105 = model_105.sampling(data=stan_data, seed=1234)
fit_105

Inference for Stan model: anon_model_acf74524cab145ec054fd26d4eb66f87.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
ab[0,0]  386.66    2.42   14.7 354.52 377.38 386.81 396.62 415.01     37   1.17
ab[1,0]  330.59    1.83  16.39 295.07 320.78 330.69  340.7 361.87     80   1.01
ab[2,0]  310.62     4.4  31.12 244.36 291.25 311.17 332.24 367.51     50    1.1
ab[3,0]  734.63   17.62 152.62  447.3 631.25 729.16 832.04 1054.1     75   1.03
ab[0,1]    7.56    0.16   0.93   5.88   6.92   7.53   8.13   9.63     34   1.18
ab[1,1]   19.74    0.14   1.25   17.3  18.92  19.74  20.58   22.3     76   1.01
ab[2,1]   12.63    0.22   1.53   9.74  11.55  12.61  13.62  15.89     50    1.1
ab[3,1]   -0.39    0.71   6.15 -13.44  -4.33  -0.15   3.82  11.07     75   1.03
ab0[0]    1.2e6   2.0e6  4.0e6 -4.6e6 -2.4e5 -2.1e4  2.3e5  1.3e7      4   2.07
ab0[1]    -

In [8]:
model_106 = pystan.StanModel('./model/model10-6.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b86505943bb9b9b84f3deadc2f3896ce NOW.


In [9]:
stan_data = {
    'N': df.shape[0],
    'K': df['KID'].nunique(),
    'X': df['X'].values,
    'Y': df['Y'].values,
    'KID': df['KID'].values,
    'Nu': 2,
}
fit_106 = model_106.sampling(data=stan_data, seed=1234)
fit_106

Inference for Stan model: anon_model_b86505943bb9b9b84f3deadc2f3896ce.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
ab[0,0]  383.54    0.37   14.6 353.75 373.98 383.54 393.47 411.71   1565    1.0
ab[1,0]  333.82    0.44  17.99 298.03 322.12 333.75 346.15 367.81   1663    1.0
ab[2,0]   329.6    0.91  33.25 258.73 307.78 332.52 353.08 389.89   1324    1.0
ab[3,0]  443.74    4.36 115.08 300.27 360.62 408.58 502.84 734.23    696    1.0
ab[0,1]    7.72    0.02    0.9   6.01   7.12    7.7   8.34   9.49   1643    1.0
ab[1,1]   19.44    0.03   1.35  16.81  18.55  19.44  20.35  22.05   1780    1.0
ab[2,1]   11.73    0.05   1.66    8.7  10.56  11.62  12.84  15.31   1341    1.0
ab[3,1]   11.26    0.18   4.66  -0.59   8.84  12.66  14.56  17.34    703    1.0
ab0[0]   369.68    1.53  59.24 257.87 339.74 363.92 391.75 518.15   1497    1.0
ab0[1]    1

In [12]:
model_107 = pystan.StanModel('./model/model10-7.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_21d94d09c8f775d17f395563fb41a98b NOW.


In [13]:
stan_data = {
    'N': df.shape[0],
    'K': df['KID'].nunique(),
    'X': df['X'].values,
    'Y': df['Y'].values,
    'KID': df['KID'].values,
    'Nu': 2,
}
fit_107 = model_107.sampling(data=stan_data, seed=1234)
fit_107

Inference for Stan model: anon_model_21d94d09c8f775d17f395563fb41a98b.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
ab[0,0]        382.57    0.36   14.7 353.05  373.3 382.59 392.37 411.24   1635    1.0
ab[1,0]        335.43    0.39  17.59 300.95 323.47 335.45 347.65 368.59   2075    1.0
ab[2,0]        329.08    0.78  32.14 263.17 308.72 331.14 351.36 387.69   1710    1.0
ab[3,0]        443.51    3.67 111.23 299.36 364.19 411.51 500.52 718.58    921    1.0
ab[0,1]          7.79    0.02   0.92   5.96   7.18   7.79   8.37   9.64   1790    1.0
ab[1,1]         19.35    0.03    1.3  16.85  18.46  19.35  20.25  21.87   2031    1.0
ab[2,1]         11.74    0.04   1.59    8.8  10.64  11.67  12.77  15.05   1777    1.0
ab[3,1]         11.28    0.15   4.48   0.01   8.98  12.42  14.47  17.37    952    1.0
ab0[0]         371.08     1.3  54.36 

In [10]:
model_108a = pystan.StanModel('./model/model10-8a.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_34332ec371933882b719312df0884127 NOW.


In [11]:
fit_108a = model_108a.sampling(seed=1234)
fit_108a

Inference for Stan model: anon_model_34332ec371933882b719312df0884127.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

         mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a        0.73    0.86   1.49  -1.99  -0.46   0.76   1.86    3.3      3   2.11
r[0]     0.03    0.04    2.4  -5.16  -0.88 2.4e-3   0.92   5.37   4000    1.0
r[1]   8.3e-3    0.04   2.31  -4.87  -0.93-2.9e-3   0.89   5.08   4000    1.0
r[2]  -2.5e-3    0.04   2.29   -5.2  -0.84   0.02   0.86   5.05   4000    1.0
r[3]     0.02    0.04   2.46  -5.35  -0.89   0.05    0.9   5.21   4000    1.0
r[4]     0.01    0.04    2.3  -5.03  -0.84 3.7e-4   0.82   5.51   4000    1.0
r[5]     0.05    0.04   2.39  -5.17  -0.88   0.04   0.92   5.38   4000    1.0
r[6]     0.03    0.04    2.3   -4.9  -0.88   0.03   0.95   4.92   4000    1.0
r[7]  -2.0e-3    0.04   2.31  -5.04  -0.87   0.02    0.9   4.82   4000    1.0
r[8]   5.7e-3    0.04   2.31   

In [14]:
model_108b = pystan.StanModel('./model/mode10-8b.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3ebc1fb38cf5614a4bf24f390f909680 NOW.


In [15]:
fit_108b = model_108b.sampling(seed=1234)
fit_108b

Inference for Stan model: anon_model_3ebc1fb38cf5614a4bf24f390f909680.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a_raw       -0.01    0.02   1.02   -2.0  -0.71  -0.02    0.7   2.01   4000    1.0
r_raw[0]     0.02    0.02    1.0  -1.97  -0.67   0.02    0.7   1.96   4000    1.0
r_raw[1]    -0.01    0.02    1.0  -1.99  -0.71  -0.03   0.69   1.98   4000    1.0
r_raw[2]     0.01    0.02   0.97  -1.91  -0.64   0.02   0.66   1.89   4000    1.0
r_raw[3]  -1.5e-4    0.02    1.0  -1.99  -0.66  -0.01   0.65   1.97   4000    1.0
r_raw[4]  -7.4e-3    0.02   0.99  -1.91  -0.69-7.1e-3   0.65   1.94   4000    1.0
r_raw[5]   4.0e-4    0.02   1.02  -2.02  -0.67 9.3e-4   0.69   2.02   4000    1.0
r_raw[6]  -6.4e-3    0.02   1.01  -2.02  -0.72 3.0e-3   0.67   1.96   4000    1.0
r_raw[7]    -0.02    0.02   0.98  -1.95   -0.7  -0.02   0.64   1.92   400

In [32]:
df = pd.read_csv('../chap08/input/data-salary-2.txt')
print(df.shape)
df.head()

(40, 3)


,X,Y,KID
0,7,457,1
1,10,482,1
2,16,518,1
3,25,535,1
4,5,427,1


In [33]:
model_84c = pystan.StanModel('./model/model8-4c.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e1dc39f1216e43d0a6e31f733a196889 NOW.


In [34]:
stan_data = {
    'N': df.shape[0],
    'K': df['KID'].nunique(),
    'X': df['X'].values,
    'Y': df['Y'].values,
    'KID': df['KID'].values,
}

fit_84c = model_84c.sampling(stan_data, seed=1234)
fit_84c

Inference for Stan model: anon_model_e1dc39f1216e43d0a6e31f733a196889.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a0       376.83    3.37  75.71 222.29 340.22 368.09 409.36 550.13    504   1.01
b0        12.23    0.21   5.45   0.35   9.39  12.28  15.18  23.82    656   1.01
a_raw[0]   0.23    0.02   0.68  -0.94  -0.24   0.15   0.64   1.77   1055    1.0
a_raw[1]  -0.42    0.02   0.63  -1.74  -0.82   -0.4  -0.02   0.73   1400    1.0
a_raw[2]  -0.49    0.02   0.67  -1.85  -0.91  -0.47  -0.07   0.84   1493    1.0
a_raw[3]    0.7    0.02   0.86  -1.05   0.14    0.7   1.26   2.41   1966    1.0
b_raw[0]  -0.63    0.02   0.64  -1.98  -1.04  -0.59  -0.18    0.5    881    1.0
b_raw[1]   0.95    0.02   0.69  -0.27   0.46    0.9   1.41   2.39    843    1.0
b_raw[2]  -0.06    0.02   0.58  -1.21  -0.46  -0.05   0.33   1.06    835    1.0
b_raw[3]  -